In [274]:
!pip install folium

In [275]:
import folium
import folium.plugins
import pandas as pd
from folium.features import *
from math import sin, cos, sqrt, atan2, radians

In [276]:
def get_distance(p0, p1):
    R = 6378

    lat1 = radians(p0[1])
    lon1 = radians(p0[0])
    lat2 = radians(p1[1])
    lon2 = radians(p1[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [277]:
def get_distance(p0, p1):
    R = 6373.0

    lat1 = radians(p0[1])
    lon1 = radians(p0[0])
    lat2 = radians(p1[1])
    lon2 = radians(p1[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return distance

In [278]:
df_cities = pd.read_csv('../../data/worldcitiespop.csv', encoding='latin-1', names=['co','name','reg_name','code','pop','lat','lon'])

In [279]:
df_cities.loc[df_cities['co']=='co'].sort_values('pop', ascending=False).head(8)

,co,name,reg_name,code,pop,lat,lon
481031,co,bogota,Bogotá,33,7102602.0,4.600000,-74.083333
481619,co,cali,Cali,29,2392897.0,3.437222,-76.522500
496938,co,medellin,Medellín,2,2000001.0,6.291389,-75.536111
480542,co,barranquilla,Barranquilla,4,1380437.0,10.963889,-74.796389
482326,co,cartagena,Cartagena,5,952036.0,10.399722,-75.514444
484770,co,cucuta,Cúcuta,21,721406.0,7.883333,-72.505278
481250,co,bucaramanga,Bucaramanga,26,571827.0,7.129722,-73.125833
499015,co,pereira,Pereira,24,440123.0,4.813333,-75.696111


In [280]:
class DivIcon(MacroElement):
    def __init__(self, html='', size=(30,30), anchor=(0,0), style=''):
        """TODO : docstring here"""
        super(DivIcon, self).__init__()
        self._name = 'DivIcon'
        self.size = size
        self.anchor = anchor
        self.html = html
        self.style = style

        self._template = Template(u"""
            {% macro header(this, kwargs) %}
              <style>
                .{{this.get_name()}} {
                    {{this.style}}
                    }
              </style>
            {% endmacro %}
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.divIcon({
                    className: '{{this.get_name()}}',
                    iconSize: [{{ this.size[0] }},{{ this.size[1] }}],
                    iconAnchor: [{{ this.anchor[0] }},{{ this.anchor[1] }}],
                    html : "{{this.html}}",
                    });
                {{this._parent.get_name()}}.setIcon({{this.get_name()}});
            {% endmacro %}
            """)

In [290]:
df_cities_M = df_cities[(df_cities['pop'] > 10000)]
lat_eu_avg = df_cities_M[df_cities_M['name'].isin(['tokio','seoul'])]['lat'].mean()
lon_eu_avg = df_cities_M[df_cities_M['name'].isin(['tokio','seoul'])]['lon'].mean()
lat_co_avg = df_cities_M[df_cities_M['co'].isin(['co'])]['lat'].mean()
lon_co_avg = df_cities_M[df_cities_M['co'].isin(['co'])]['lon'].mean()
print(lat_co_avg, lon_co_avg, lat_eu_avg, lon_eu_avg)
diff_lat = lat_co_avg - lat_eu_avg
diff_lon = lon_co_avg - lon_eu_avg

#df_cities_M['lon'] = df_cities_M['lon'] - diff_lon + 3.2
#df_cities_M['lat'] = df_cities_M['lat'] - diff_lat + 20.5
df_cities_M['lon'] = df_cities_M['lon'] - diff_lon + 1.2
df_cities_M['lat'] = df_cities_M['lat'] - diff_lat + 0.5

d_names = {
    'quibdo':'Quibdó',
    'bogota':'Bogotá',
    'cali':'Cali',
    'cartagena':'Cartagena',
    'cucuta':'Cúcuta',
    'leticia':'Leticia',
    'riohacha':'Riohacha'
}
spa_name = lambda x:x if (not x in d_names) else d_names[x] 

df_cities_M = df_cities_M.loc[df_cities_M['name'].isin(list(d_names))]
df_cities_M = df_cities_M.loc[df_cities_M['lat']< 50]
df_cities_M = df_cities_M.reset_index(drop=True)
df_cities_M = df_cities_M.sort_values('lat')

6.561388888888889 -74.95991898576388 37.5663889 126.99972220000001


<ipython-input-290-7f42512a7991>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cities_M['lon'] = df_cities_M['lon'] - diff_lon + 1.2
<ipython-input-290-7f42512a7991>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cities_M['lat'] = df_cities_M['lat'] - diff_lat + 0.5


In [292]:
m = folium.Map([lat_eu_avg, lon_eu_avg],
                zoom_start=5)

folium.TileLayer('cartodbpositron').add_to(m)

for city in df_cities_M.iterrows():
    #City
    lat = city[1]['lat']
    lon = city[1]['lon']
    name = city[1]['name']
    if name in d_names:
        folium.map.Marker(
            [lat,lon],icon=folium.Icon(icon='star',color='orange')).add_to(m)
        folium.map.Marker(
            [lat,lon],
            icon=DivIcon(
                size=(150,36),
                anchor=(150,0),
                html=spa_name(name),
                style="""
                    font-size:12px;
                    background-color: transparent;
                    border-color: transparent;
                    text-align: right;
                    """
                )
            ).add_to(m)
        
#folium.PolyLine([[df_cities_M['lat'].iloc[0],df_cities_M['lon'].iloc[0]], [df_cities_M['lat'].iloc[1],df_cities_M['lon'].iloc[1]]], color="red", weight=2.5, opacity=1).add_to(m)

l = list(df_cities_M.iterrows())
i_ant = l[0]
for i in l[1:]:
    folium.PolyLine([[i_ant[1]['lat'],i_ant[1]['lon']], [i[1]['lat'],i[1]['lon']]], color="red", weight=2.5, opacity=1).add_to(m)
    #folium.map.Marker(
    #[(i_ant[1]['lat']+i[1]['lat'])/2,(i_ant[1]['lon']+i[1]['lon'])/2+4.5],
    #icon=DivIcon(
    #    size=(150,36),
    #    anchor=(150,0),
    #    html="%s km" % int(get_distance((i_ant[1]['lat'],i_ant[1]['lon']),(i[1]['lat'],i[1]['lon']))),
    #    style="""
    #        font-size:16px;
    #        background-color: transparent;
    #        border-color: transparent;
    #        text-align: right;
    #        """
    #    )
    #)
    #.add_to(m)
    i_ant = i

m
m.save('/Users/jfm/map.html')

In [283]:
l

[(4, co               co
  name        leticia
  reg_name    Leticia
  code              1
  pop           29666
  lat         47.2897
  lon         135.219
  Name: 4, dtype: object), (1, co                  co
  name              cali
  reg_name          Cali
  code                29
  pop         2.3929e+06
  lat            54.9422
  lon            128.637
  Name: 1, dtype: object), (0, co                  co
  name            bogota
  reg_name        Bogotá
  code                33
  pop         7.1026e+06
  lat             56.105
  lon            131.076
  Name: 0, dtype: object), (5, co               co
  name         quibdo
  reg_name     Quibdó
  code             11
  pop           75105
  lat         57.1997
  lon         128.499
  Name: 5, dtype: object), (3, co               co
  name         cucuta
  reg_name     Cúcuta
  code             21
  pop          721406
  lat         59.3883
  lon         132.654
  Name: 3, dtype: object), (2, co                 co
  name        ca